In [1]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.parquet('./ticket_preprocessed.parquet')
df.show()
df.count()

+--------+-------------------+----------------+----------+--------+----------+------------------+----------+----------+----------+------------------+----------+----------+--------+
|班次代码|           发车时间|      乘车站名称|到达站名称|座位类型|乘车站省份|乘车站行政地理分区|乘车站经度|乘车站纬度|到达站省份|到达站行政地理分区|到达站经度|到达站纬度|    距离|
+--------+-------------------+----------------+----------+--------+----------+------------------+----------+----------+----------+------------------+----------+----------+--------+
|  KS1057|2020-05-07 09:00:00|苏州北广场汽车站|  常熟南站|       1|    江苏省|              华东|120.608475| 31.330946|    江苏省|              华东| 120.74239| 31.628862|  35.478|
|  KS3197|2020-05-07 10:50:00|        苏州南站|  常熟南站|       1|    江苏省|              华东|120.638145|  31.27728|    江苏省|              华东| 120.74239| 31.628862|  40.325|
|  GT1001|2020-05-07 17:40:00|          沙溪站|    太仓站|       1|    湖南省|              华中|109.902885| 26.756468|    江苏省|              华东| 121.19665| 31.510124| 1216.72|
|  GT1001|2020-05-07 17:40:00|          沙溪

38448537

# 提取桑基图数据

In [9]:
#地理区域分区
sections = {
    '华北': ['北京市', '天津市', '河北省', '山西省', '内蒙古自治区'],
    '东北': ['黑龙江省', '吉林省', '辽宁省'],
    '华东': ['上海市', '江苏省', '浙江省', '安徽省', '江西省', '山东省', '福建省', '台湾省'],
    '华中': ['河南省', '湖北省', '湖南省'],
    '华南': ['广东省', '广西壮族自治区', '海南省', '香港特别行政区', '澳门特别行政区'],
    '西南': ['重庆市', '四川省', '贵州省', '云南省', '西藏自治区'],
    '西北': ['陕西省', '甘肃省', '青海省', '宁夏回族自治区', '新疆维吾尔自治区']
}

inv_sections = {province: section for section, provinces in sections.items() for province in provinces}

In [53]:
#以地理区域划分为单位进行分组统计
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
inflow = df[df.乘车站省份 != df.到达站省份].groupBy('到达站省份','乘车站省份').count().toPandas()
inflow.columns = ['到达站省份', '乘车站省份','人次']
inflow['到达站行政地理分区'] = inflow['到达站省份'].map(lambda x: inv_sections[x])
inflow['乘车站行政地理分区'] = inflow['乘车站省份'].map(lambda x: inv_sections[x])
inflow = inflow.groupby(by = ['到达站行政地理分区','乘车站行政地理分区']).sum()
inflow.to_csv('sankey_gbk2.csv',encoding='gbk')#gbk编码的格式
inflow.to_csv('sankey2.csv')#默认的utf编码保存
inflow.head()

人次
到达站行政地理分区 乘车站行政地理分区        
东北        东北         397282
          华东         212235
          华中          57213
          华北          40527
          华南          19416

In [4]:
#按到达站和乘车站分组统计，统计各个省份间的流动情况
inflow = df[df.乘车站省份 != df.到达站省份].groupBy('到达站省份','乘车站省份').count().toPandas()
inflow.head()
inflow.to_csv('sankey_gbk.csv',encoding='gbk')#gbk编码的格式
inflow.to_csv('sankey.csv')#默认的utf编码保存

,到达站省份,乘车站省份,count
0,甘肃省,江苏省,33201
1,重庆市,浙江省,1149
2,海南省,广西壮族自治区,667
3,广西壮族自治区,河南省,1231
4,吉林省,安徽省,38080


In [55]:
#这部分是按照地理行政区域的方位顺序重新排一下序
# 设置成“category”数据类型
df = pd.read_csv('sankey2.csv')
df['乘车站行政地理分区'] = df['乘车站行政地理分区'].astype('category')
# inplace = True，使 recorder_categories生效
list_custom = ['东北','华北','华中','华东','华南','西南','西北']
df['乘车站行政地理分区'].cat.reorder_categories(list_custom, inplace=True)


# inplace = True，使 df生效

df.sort_values('乘车站行政地理分区', inplace=True)

df.head()

,到达站行政地理分区,乘车站行政地理分区,人次
0,东北,东北,397282
42,西南,东北,108542
35,西北,东北,84405
28,华南,东北,63303
7,华东,东北,360109


In [56]:
inflow.to_csv('sankey_gbk2.csv',encoding='gbk')#gbk编码的格式
inflow.to_csv('sankey2.csv')#默认的utf编码保存

In [ ]:
def get_provincial_flows(df):
    
    inflow = df[df.乘车站省份 != df.到达站省份].groupBy('到达站省份').count().toPandas()
    inflow.columns = ['省份', '流入人次']

    outflow = df[df.乘车站省份 != df.到达站省份].groupBy('乘车站省份').count().toPandas()
    outflow.columns = ['省份', '流出人次']

    internal_flow = df[df.乘车站省份 == df.到达站省份].groupBy('乘车站省份').count().toPandas()
    internal_flow.columns = ['省份', '内部流动人次']
    
    provinces = inflow.merge(outflow, how='outer').merge(internal_flow, how='outer').fillna(0)
    provinces[['流出人次', '内部流动人次']] = provinces[['流出人次', '内部流动人次']].astype(int)
    provinces.set_index('省份', inplace=True)

    return provinces

## 计算特定时段的人口流动

### 限制一定时间范围内对各个省流动情况统计

In [6]:
#定义好的流动情况统计函数
def get_provincial_flows(df):
    
    inflow = df[df.乘车站省份 != df.到达站省份].groupBy('到达站省份').count().toPandas()
    inflow.columns = ['省份', '流入人次']

    outflow = df[df.乘车站省份 != df.到达站省份].groupBy('乘车站省份').count().toPandas()
    outflow.columns = ['省份', '流出人次']

    internal_flow = df[df.乘车站省份 == df.到达站省份].groupBy('乘车站省份').count().toPandas()
    internal_flow.columns = ['省份', '内部流动人次']
    
    provinces = inflow.merge(outflow, how='outer').merge(internal_flow, how='outer').fillna(0)
    provinces[['流出人次', '内部流动人次']] = provinces[['流出人次', '内部流动人次']].astype(int)
    provinces.set_index('省份', inplace=True)

    return provinces

In [42]:
#限定时间范围得到新的表
df_1 = df[(df.发车时间 >= '2020-06-01') & (df.发车时间 < '2020-07-01')] 
df_1 = df_1[df.乘车站省份=='湖北省']
inflow = df_1[df_1.乘车站省份 != df_1.到达站省份].groupBy('到达站省份').count().toPandas()
inflow.columns = ['省份', '流出人次']
inflow

,省份,流出人次
0,西藏自治区,19
1,北京市,362
2,辽宁省,1136
3,浙江省,1481
4,广西壮族自治区,431
5,海南省,18
6,重庆市,1749
7,河北省,1970
8,福建省,1078
9,湖南省,3342


In [ ]:
provinces = get_provincial_flows(df_1)
provinces['行政地理分区'] = provinces.index.map(lambda x: inv_sections[x])
provinces.to_csv('provincial_flows_06.csv')
provinces

### 清明节前后

In [10]:
df_1 = df[(df.发车时间 >= '2020-04-04') & (df.发车时间 < '2020-04-07')] 

In [11]:
provinces = get_provincial_flows(df_1)
provinces['行政地理分区'] = provinces.index.map(lambda x: inv_sections[x])
provinces.to_csv('provincial2020.csv')
provinces

,流入人次,流出人次,内部流动人次,行政地理分区
省份,,,,
西藏自治区,287,3680,18195,西南
北京市,1551,96,0,华北
辽宁省,9333,19260,50860,东北
浙江省,19204,59190,517839,华东
广西壮族自治区,21661,436,978,华南
海南省,2466,574,21196,华南
重庆市,6586,187,0,西南
香港特别行政区,160,0,0,华南
河北省,4461,506,0,华北


In [12]:
df_1 = df[(df.发车时间 >= '2021-04-03') & (df.发车时间 < '2021-04-06')] 

In [13]:
provinces = get_provincial_flows(df_1)
provinces['行政地理分区'] = provinces.index.map(lambda x: inv_sections[x])
provinces.to_csv('provincial2021.csv')
provinces

,流入人次,流出人次,内部流动人次,行政地理分区
省份,,,,
西藏自治区,673,502,15635,西南
北京市,3081,58,0,华北
辽宁省,8637,52688,221271,东北
浙江省,14413,1452,781,华东
广西壮族自治区,9996,1677,7417,华南
海南省,581,81,0,华南
重庆市,6298,100,0,西南
香港特别行政区,2260,0,0,华南
河北省,8185,53,2,华北


### 2021年2～7月

In [11]:
df_1 = df[(df.发车时间 >= '2020-02-01') & (df.发车时间 < '2020-07-01')] 

In [12]:
provinces = get_provincial_flows(df_1)
provinces['行政地理分区'] = provinces.index.map(lambda x: inv_sections[x])
provinces.to_csv('../../数据/provincial_flows_1.csv')
provinces

,流入人次,流出人次,内部流动人次,行政地理分区
省份,,,,
西藏自治区,15543,29667,518929,西南
北京市,55507,1109,2,华北
辽宁省,239272,773623,2207806,东北
浙江省,488617,770677,5910976,华东
广西壮族自治区,165728,12629,38914,华南
海南省,42608,17886,500617,华南
重庆市,224305,11493,16265,西南
香港特别行政区,5435,0,0,华南
河北省,157311,16101,1,华北


In [13]:
sections = provinces.groupby('行政地理分区').agg('sum')
sections.to_csv('../../数据/sectional_flows_1.csv')
sections

,流入人次,流出人次,内部流动人次
行政地理分区,,,
东北,814438,1089444,2879432
华东,2450021,2336036,12794253
华中,469079,597393,916505
华北,541632,75668,174902
华南,470774,193816,1554371
西北,498612,557090,3156073
西南,654034,1049143,4107806


### 2021年3～4月

In [14]:
df_2 = df[(df.发车时间 >= '2021-03-01') & (df.发车时间 < '2021-05-01')]

In [15]:
provinces = get_provincial_flows(df_2)
provinces['行政地理分区'] = provinces.index.map(lambda x: inv_sections[x])
provinces.to_csv('../../数据/provincial_flows_2.csv')
provinces

,流入人次,流出人次,内部流动人次,行政地理分区
省份,,,,
西藏自治区,2969,2959,125087,西南
北京市,15428,275,0,华北
辽宁省,47830,217355,882940,东北
浙江省,118908,9661,8470,华东
广西壮族自治区,49720,5591,27745,华南
海南省,4469,187,0,华南
重庆市,54268,1148,0,西南
香港特别行政区,13795,0,0,华南
河北省,44869,346,84,华北


In [16]:
sections = provinces.groupby('行政地理分区').agg('sum')
sections.to_csv('../../数据/sectional_flows_2.csv')
sections

,流入人次,流出人次,内部流动人次
行政地理分区,,,
东北,164942,226314,896214
华东,469885,209820,690530
华中,130637,509505,1273802
华北,155375,1020,282
华南,175453,68238,298205
西北,103830,98436,589621
西南,169862,256651,1100297
